In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import torch
# torch.ones(1,2).cuda()

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

STITCH_BATCH_SIZE = 32 # todo study the effect
EVAL_BATCH_SIZE = 64
MAX_DEPTH = 10
THRESOULD = 0.9
TOTAL_THRESOULD = 0.7

RESULT_NAME = "result_paper_32"

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [5]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [ ]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score)
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'w') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False)
    for i,(s,net) in enumerate(generator):
        netname = f"_results/{RESULT_NAME}/net{k:03}_{i:03}"
        report.evaluate(nets, net, netname, s, dataset_val)
        net.save(netname)
        k += 1

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


current depth: 1


100%|██████████| 5/5 [00:13<00:00,  2.73s/it]


potential next fragments: 43
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0
epoch 0 loss 4.740804635007105e-10
epoch 1 loss 6.009904279768357e-11
epoch 2 loss 1.4016764924475861e-11
epoch 3 loss 7.914613923091189e-12
epoch 4 loss 6.083193219842852e-12
epoch 5 loss 5.152082159770489e-12
epoch 6 loss 4.53993493842612e-12
epoch 7 loss 4.123908918969719e-12
epoch 8 loss 3.84631859208276e-12
epoch 9 loss 3.652837748786516e-12
current depth: 2


100%|██████████| 5/5 [00:08<00:00,  1.74s/it]


...

  0%|          | 0/80 [00:00<?, ?it/s]/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 80/80 [01:36<00:00,  1.20s/it]


len(self.fragments) 1
saving to _results/result_paper_32/net002_000
totalscore 0.8411592592743347
epoch 0 loss 0.0
current depth: 8


100%|██████████| 5/5 [00:04<00:00,  1.19it/s]


potential next fragments: 43
potential next fragments after thresholding of 0.9: 2 ['0.92', '0.9']
totalscore 0.7755337883304582
epoch 0 loss 0.0


  0%|          | 0/80 [00:00<?, ?it/s]/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
 78%|███████▊  | 62/80 [01:26<00:25,  1.40s/it]